<a href="https://colab.research.google.com/github/cat0ros/robotics-control-3DOF/blob/master/upgrade_fk_ik_python_3dof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание по управлению роботами №1
Выполнил: Романов Ростислав, группа: АДБ-20-09

<img src= "https://drive.google.com/uc?export=view&id=1PpzfCIzka8X_ljyS4piV1AKeBfmu2AZ3" alt="inverse" width="400" />

***Рисунок 1: Пространственное представление схемы манипулятора***

<img src= "https://drive.google.com/uc?export=view&id=1zDL-aUGP3qxmC3ibMRZ7z9wZ6hDe7knu" alt="inverse" width="400" />

***Рисунок 2: Кинематическая схема манипулятора***

## Параметры и описание робота
Длины звеньев робота:

In [ ]:
links_length = [1.5, 1.3, 2.2]

## Библиотеки для работы

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from numpy.linalg import inv
import math
import sympy as sp
from IPython.display import HTML, Math, Latex
%matplotlib notebook

## Решение прямой задачи кинематики
**Составление DH-матриц**

Матрица Денавита-Хартенберга:

<img src= "https://wikimedia.org/api/rest_v1/media/math/render/svg/6963d0c47a3a894ff0719c8df348d188b996074e" alt="inverse" width="400" />

Функции вычисления матриц преобразований:

In [ ]:
def rz(a):
    return sp.Matrix([
        [sp.cos(a), -sp.sin(a), 0, 0],
        [sp.sin(a), sp.cos(a), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def ry(a):
    return sp.Matrix([
        [sp.cos(a), 0, sp.sin(a), 0],
        [0, 1, 0, 0],
        [-sp.sin(a), 0, sp.cos(a), 0],
        [0, 0, 0, 1]
    ])

def rx(a):
    return sp.Matrix([
        [1, 0, 0, 0],
        [0, sp.cos(a), -sp.sin(a), 0],
        [0, sp.sin(a), sp.cos(a), 0],
        [0, 0, 0, 1]
    ])

def trs(x, y, z):
    return sp.Matrix([
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

def vec(x, y, z):
    return sp.Matrix([
        [x],
        [y],
        [z],
        [1]
    ])

def dh(theta, d, alpha, a):
    return rz(theta) * trs(0, 0, d) * rx(alpha) * trs(a, 0, 0)


Реализуем функцию, которая будет возвращать матрицу под соответствующие DH-параметры:

Расставим системы координат и составим таблицу, а по ней итоговую матрицу преобразований:

<img src= "https://drive.google.com/uc?export=view&id=18EiOZVcCKHVvgkmfPqdece4xYVxcEvYH" alt="inverse" width="600" />

Заметим, что нулевое положение робота будет слева, тогда для изображения справа повернем вокруг оси Z концевую точку на 90 градусов в положительном направлении.

In [ ]:
def forward_kinematics(generilized, lengths, ret_all_dh=False):
  theta1, d2, theta3 = generilized
  len1, len2, len3 = lengths[0], lengths[1], lengths[2]
  dh1 = dh(theta1 - sp.pi / 2, len1, -sp.pi / 2, 0)
  dh2 = dh(sp.pi / 2, d2 + len2, sp.pi / 2, 0)
  dh3 = dh(theta3 + sp.pi / 2, 0, 0, len3)

  dh_op = dh1 * dh2 * dh3
  if ret_all_dh:
    return dh_op, dh1, dh2, dh3
  else:
    return dh_op

Результирующая матрица для данной кинематической схемы:

In [ ]:
my_generilizeds = (sp.symbols("theta_1"), sp.symbols("d_2"), sp.symbols("theta_3"))
lengths_sym = [sp.symbols("l_1"), sp.symbols("l_2"), sp.symbols("l_3")]
fk_syms_sol, fk_sym_dh1, fk_sym_dh2, fk_sym_dh3 = forward_kinematics(my_generilizeds, lengths_sym, True)

fk_syms_sol

Matrix([
[cos(theta_1)*cos(theta_3), -sin(theta_3)*cos(theta_1),  sin(theta_1), l_3*cos(theta_1)*cos(theta_3) + (d_2 + l_2)*cos(theta_1)],
[sin(theta_1)*cos(theta_3), -sin(theta_1)*sin(theta_3), -cos(theta_1), l_3*sin(theta_1)*cos(theta_3) + (d_2 + l_2)*sin(theta_1)],
[             sin(theta_3),               cos(theta_3),             0,                                   l_1 + l_3*sin(theta_3)],
[                        0,                          0,             0,                                                        1]])

Первая матрица преобразований:

In [ ]:
sp.simplify(fk_sym_dh1)

Matrix([
[ sin(theta_1),  0, cos(theta_1),   0],
[-cos(theta_1),  0, sin(theta_1),   0],
[            0, -1,            0, l_1],
[            0,  0,            0,   1]])

Вторая матрица преобразований:

In [ ]:
sp.simplify(fk_sym_dh2)

Matrix([
[0, 0, 1,         0],
[1, 0, 0,         0],
[0, 1, 0, d_2 + l_2],
[0, 0, 0,         1]])

Третья матрица преобразований:

In [ ]:
sp.simplify(fk_sym_dh3)

Matrix([
[-sin(theta_3), -cos(theta_3), 0, -l_3*sin(theta_3)],
[ cos(theta_3), -sin(theta_3), 0,  l_3*cos(theta_3)],
[            0,             0, 1,                 0],
[            0,             0, 0,                 1]])

Решим ПЗК для вектора обобщенных координат q = [30, 2.2, 90]

In [ ]:
gens_test = (np.deg2rad(30), 2.2, np.deg2rad(90))
fk_test_sol = forward_kinematics(gens_test, links_length)
fk_test_sol

Matrix([
[5.30287619362453e-17,   -0.866025403784439,                0.5, 3.03108891324554],
[3.06161699786838e-17,                 -0.5, -0.866025403784439,             1.75],
[                 1.0, 6.12323399573677e-17,                  0,              3.7],
[                   0,                    0,                  0,                1]])

# Решение обратной задачи кинематики

Рассмотрид вид сверху исходной кинематической схемы:

<img src= "https://drive.google.com/uc?export=view&id=16u0Gn5cK_Mwyt6NhC_CyctTbbU0kwyRr" alt="inverse" width="200" />

Заметим следующую зависимость:

$$\Theta_{1} = \text{arctan2}(y_0, x_0)$$

Не является возможным найти сразу решение для второй обобщенной координаты, так как нам неизвестна проекция длины третьего звена.

Найдем ее по виду сбоку нашего манипулятора и найдем высоту обозначенную как b:

<img src= "https://drive.google.com/uc?export=view&id=1oQSe0p5Er6_KpYB8kVftatvfdA_2yEO7" alt="inverse" width="400" />

$$b = z - l_{1}$$

Тогда становится известна проекция длины третьего звена на плоскость X0Y0:

$$l_{3_{x_{0}y_{0}}} = l_{3}*cos(\Theta_{3})$$

Найдем этот угол:

$$\Theta_{3} = arcsin(\frac{z - l1}{l3})$$

$$\Theta_{3} = \pi - arcsin(\frac{z - l1}{l3})$$

В результате мы получили два решения.

Так как от найденного угла зависит проекция третьего звена на плоскость X0Y0, вычислим вторую обобщенную координату:

$$d2 = \sqrt{x_{0}^{2} + y_{0}^{2}} - l_{2} - l_{3} * cos(\Theta_{3})$$

Аналитические выражения для решения ОЗК получены.

In [ ]:
def IK(coordinates, lengths):
  x, y, z = coordinates
  l1, l2, l3 = lengths[0], lengths[1], lengths[2]
  theta1 = sp.atan2(y, x)
  theta3_1 = sp.asin((z - l1) / l3)
  theta3_2 = sp.pi - sp.asin((z - l1) / l3)
  theta3_sol = [theta3_1, theta3_2]
  sqrt_xy = (x**2 + y**2)**(1/2)
  l3_x0y0_1 = l3 * sp.cos(theta3_1)
  l3_x0y0_2 = l3 * sp.cos(theta3_2)
  d2_1 = sqrt_xy - l3_x0y0_1 - l2
  d2_2 = sqrt_xy - l3_x0y0_2 - l2
  d2_sol = [d2_1, d2_2]

  for i in range(len(theta3_sol)):
    q = [theta1, d2_sol[i], theta3_sol[i]]
    yield q

Решим прямую задачу кинематики для обобщенных координат: [30, 2.2, 90]

In [ ]:
coords_test = (np.deg2rad(30), 2.2, np.deg2rad(90))
fk_sol = forward_kinematics(coords_test, links_length)
fk_sol

Matrix([
[5.30287619362453e-17,   -0.866025403784439,                0.5, 3.03108891324554],
[3.06161699786838e-17,                 -0.5, -0.866025403784439,             1.75],
[                 1.0, 6.12323399573677e-17,                  0,              3.7],
[                   0,                    0,                  0,                1]])

Вычислим позицию концевой точки, домножив на вектор-столбец: [0,0,0,1]

In [ ]:
tcp_pos = fk_sol * vec(0, 0, 0)
tcp_pos

Matrix([
[3.03108891324554],
[            1.75],
[             3.7],
[               1]])

In [ ]:
x_f, y_f, z_f = tcp_pos[0, 0], tcp_pos[1, 0], tcp_pos[2, 0]

Переберем все решения ОЗК для данной точки и запишем их. Решения ОЗК должны совпать с тем, что мы задавали вначале в функции ПЗК в качестве обобщенных координат. Задавали вектор: [30, 2.2, 90]

In [ ]:
coords = (x_f, y_f, z_f)
list_sols = [i for i in IK(coords, links_length)]
for i in list_sols:
  q1, q3 = sp.deg(i[0]), sp.deg(i[2])
  print(q1.evalf(), i[1], q3.evalf())

30.0000000000000 2.20000000000000 90.0000000000000
30.0000000000000 2.20000000000000 90.0000000000000


Пометка: для угла 90 градусов решение ОЗК в данном случае однозначно.

Проверка выполнена.

Подставим позицию конкретной точки и вычислим для нее ОЗК (в соответствии с вариантом): [0.4,0.3, 0.1]

In [ ]:
hw_coords = (0.4, 0.3, 0.1)

IK_variants = [i for i in IK(hw_coords, links_length)]
for i in IK_variants:
  q1, q3 = sp.deg(i[0]), sp.deg(i[2])
  print(q1.evalf(), i[1], q3.evalf())

36.8698976458440 -2.49705627484771 -39.5211963586422
36.8698976458440 0.897056274847714 219.521196358642


# Прямая задача по скорости

Вычислим линейные составляющие скорости. Найдем частные производные и составим матрицу.

In [ ]:
def Jv_calc(fk_solved, gens):
  gripper_pose = fk_solved * vec(0, 0, 0)
  x, y, z = gripper_pose[0], gripper_pose[1], gripper_pose[2]
  coords = (x, y, z)
  jacobi_matrix = sp.Matrix([[0,0,0],[0,0,0],[0,0,0]])
  for i in range(len(coords)):
    for j in range(len(gens)):
      jacobi_matrix[i, j] = sp.diff(coords[i], gens[j])

  return jacobi_matrix

In [ ]:
Jv_ = Jv_calc(fk_syms_sol, my_generilizeds)

Для вычисления угловой составляющей известно, что компонента для второго сочленения будет представлена вектор-столбцом состоящим из нулей.


In [ ]:
dw2 = sp.Matrix([[0], [0], [0]])

Компонента первой степени подвижности будет равна:



In [ ]:
dw1 = sp.eye(3, 3) * sp.Matrix([[0], [0], [1]])

In [ ]:
dw1

Matrix([
[0],
[0],
[1]])

Компонента третей степени подвижности:

Найдем матрицу поворота второй системы координат (третей степени подвижности) относительно нулевой (известна на этапе решения ПЗК)

In [ ]:
rm2 = fk_sym_dh1 * fk_sym_dh2
rotate_m2 = rm2[0:3, 0:3]
rotate_m2

Matrix([
[ 0, cos(theta_1),  sin(theta_1)],
[ 0, sin(theta_1), -cos(theta_1)],
[-1,            0,             0]])

Составим итоговую матрицу Якоби:

In [ ]:
Jtotal = sp.Matrix([
    [Jv_],
    [dw1, dw2, rotate_m2 * sp.Matrix([[0],[0],[1]])]
])

In [ ]:
Jtotal

Matrix([
[-l_3*sin(theta_1)*cos(theta_3) - (d_2 + l_2)*sin(theta_1), cos(theta_1), -l_3*sin(theta_3)*cos(theta_1)],
[ l_3*cos(theta_1)*cos(theta_3) + (d_2 + l_2)*cos(theta_1), sin(theta_1), -l_3*sin(theta_1)*sin(theta_3)],
[                                                        0,            0,               l_3*cos(theta_3)],
[                                                        0,            0,                   sin(theta_1)],
[                                                        0,            0,                  -cos(theta_1)],
[                                                        1,            0,                              0]])

## Обратная задача кинематики в скоростях

Найдем матрицу обратную матрице Якоби линейных скоростей:

In [ ]:
Jv_inv = sp.simplify(Jv_.inv())
Jv_inv

Matrix([
[-sin(theta_1)/(d_2 + l_2 + l_3*cos(theta_3)), cos(theta_1)/(d_2 + l_2 + l_3*cos(theta_3)),                    0],
[                                cos(theta_1),                                sin(theta_1),         tan(theta_3)],
[                                           0,                                           0, 1/(l_3*cos(theta_3))]])

In [ ]:
Vels = sp.Matrix([
    [sp.symbols('V_x')],
    [sp.symbols('V_y')],
    [sp.symbols('V_z')]
])

Общее решение ОЗК по скорости:

In [ ]:
ik_vels = sp.simplify(Jv_inv * Vels)
ik_vels

Matrix([
[(-V_x*sin(theta_1) + V_y*cos(theta_1))/(d_2 + l_2 + l_3*cos(theta_3))],
[               V_x*cos(theta_1) + V_y*sin(theta_1) + V_z*tan(theta_3)],
[                                               V_z/(l_3*cos(theta_3))]])

Подставим решение ОЗК по варианту и скорости по варианту:

In [ ]:
jv_sols = []
for i in IK_variants:
  Jv_inv__ = sp.Matrix(
      [
          [(sp.sin(sp.rad(i[0])) / (i[1] + links_length[1] + links_length[2] * sp.cos(sp.rad(i[2])))),
          sp.cos(sp.rad(i[0])) / (i[1] + links_length[1] +links_length[2] * sp.cos(sp.rad(i[2]))),
          0],
          [
            sp.cos(sp.rad(i[0])),  sp.sin(sp.rad(i[0])), sp.sin(sp.rad(i[2])) / sp.cos(sp.rad(i[0]))
          ],
          [0, 0, 1 / (links_length[2] * sp.cos(sp.rad(i[2])))]
      ]
  )
  jv_sols.append(Jv_inv__)

In [ ]:
jv_sols[1].evalf()

Matrix([
[0.00255706298634515,    0.2276651194445,                  0],
[  0.999936930589248, 0.0112309769723537, 0.0668243702343456],
[                  0,                  0,  0.455563622903216]])

In [ ]:
Vel_var = sp.Matrix([
    [-0.1],
    [0.5],
    [0]
])

Ответ на ОЗК по скорости:

In [ ]:
(jv_sols[1] * Vel_var).evalf()

Matrix([
[  0.113576853423616],
[-0.0943782045727479],
[                  0]])

In [ ]:
(jv_sols[0] * Vel_var).evalf()

Matrix([
[  0.497460288957802],
[-0.0943782045727479],
[                  0]])